## 1. Import libraries and data

In [1]:
import pandas as pd
import numpy as np

merged = pd.read_csv("./data/exported/merged.csv")
score = pd.read_csv("./data/score.csv")

## 2. Merge the merged df with score.csv

> we need to do this, so we can see which applicant_id matches with which applicant

Checking if shapes are same length

In [2]:
print(merged.shape)
print(score.shape)

(500, 5)
(500, 50)


In [3]:
score.head(3)

,income_am,profit_last_am,profit_am,damage_am,damage_inc,crd_lim_rec,credit_use_ic,gluten_ic,lactose_ic,insurance_ic,...,score1_pos,score1_neg,score2_pos,score2_neg,score3_pos,score3_neg,score4_pos,score4_neg,score5_pos,score5_neg
0,5660.0,4320.0,8640.0,0.0,0.0,8000.0,0.0,0.0,1.0,0.0,...,0.538419,0.396819,0.423742,0.763608,NaN,NaN,NaN,NaN,NaN,NaN
1,3990.0,9.0,3450.0,0.0,0.0,12500.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1158.0,82.0,4194.0,408.0,4.0,12000.0,0.0,0.0,0.0,1.0,...,0.009811,0.592842,NaN,NaN,0.252444,0.724693,0.818064,0.387361,NaN,NaN


In [4]:
merged.head(3)

,applicant_id,predicted_damage_amount,predicted_damage_incident,predicted_revenue,predicted_profit
0,0,0.000000,0,1039.082571,1039.082571
1,1,0.000000,0,2597.720755,2597.720755
2,2,385.169337,1,1579.118807,1193.949471


In every model's notebook we do this code ...

``` python
applicants_data["predicted_XXX"] = XXX
applicants_data["applicant_id"] = applicants_data.index
```

... Which means the applicant_id is the same as the index. So we can just merge on index

In [5]:
merged_with_score = pd.merge(merged, score, left_index=True, right_index=True, how="left")

### **Testing if it merged correctly**

In [6]:
merged_check = merged_with_score[["income_am", "profit_am", "applicant_id", "predicted_profit"]]
merged_check

,income_am,profit_am,applicant_id,predicted_profit
0,5660.0,8640.0,0,1039.082571
1,3990.0,3450.0,1,2597.720755
2,1158.0,4194.0,2,1193.949471
3,2451.0,2119.0,3,2496.358789
4,946.0,2036.0,4,1626.727706
...,...,...,...,...
495,820.0,7794.0,495,2674.283195
496,6092.0,3137.0,496,4365.196865
497,2301.0,2516.0,497,1869.162869
498,492.0,3716.0,498,1956.991603


APPLICANT 100

In [7]:
merged.iloc[100]

applicant_id                  100.000000
predicted_damage_amount         0.000000
predicted_damage_incident       0.000000
predicted_revenue            2182.753611
predicted_profit             2182.753611
Name: 100, dtype: float64

In [8]:
score.iloc[100][["income_am", "profit_am"]]

income_am    122993.0
profit_am     42142.0
Name: 100, dtype: object

In [9]:
merged_check.iloc[100]

income_am           122993.000000
profit_am            42142.000000
applicant_id           100.000000
predicted_profit      2182.753611
Name: 100, dtype: float64

## 3. Get the scores of all applicants

I considered dropping all applicants with negative net scores (they're not good applicants, because their negativity-score outweighs their positivity-score). However, when I calculated the total predicted profit we came to `$388,171.26`, and if we do **not** drop the applicants with negative net scores the total predicted profit is `$499,209.45` (predicted profits for both scenarios shown below).

In best interest of the hotel, I will pursue higher profits and thus not drop applicants with negative net scores

In [10]:
import sys
sys.path.append("./util/")

from calculate_scores import calculate_scores

Calculate the scores for each applicant

In [11]:
merged_calculated_scores = calculate_scores(merged_with_score.copy())
merged_calculated_scores["net_score"] = \
    merged_calculated_scores["combined_pos_score"] - merged_calculated_scores["combined_neg_score"]

~~Only keep applicants with positive `net_score`s~~

In [12]:
# merged_calculated_scores = merged_calculated_scores[merged_calculated_scores["net_score"] >= 0]
# merged_calculated_scores.shape

### TESTING: if dropping negative applicants benefits the hotel

In [13]:
print("\n -- best 200 -- \n", 

# just taking the best 200 in terms of predicted profit
    merged_calculated_scores \
            .sort_values(by="predicted_profit", ascending=False) \
            .head(200) \
            [["predicted_profit"]].sum()
)


print("\n -- best 200, only positive score -- \n", 

# only taking the ones with a positive net score, 
# and then the best 200 in terms of predicted profit
    merged_calculated_scores[merged_calculated_scores["net_score"] >= 0] \
        .sort_values(by="predicted_profit", ascending=False) \
        .head(200) \
        [["predicted_profit"]].sum()
)


 -- best 200 -- 
 predicted_profit    499209.454366
dtype: float64

 -- best 200, only positive score -- 
 predicted_profit    388171.255842
dtype: float64


## 4. Get the 200 guests with the highest predicted profit

In [14]:
highest_profit = merged_calculated_scores \
    .sort_values(by="predicted_profit", ascending=False) \
    .head(200)

highest_profit_unsorted = highest_profit \
    .sort_values(by="applicant_id", ascending=True)

highest_profit_unsorted.head(40)

,applicant_id,predicted_damage_amount,predicted_damage_incident,predicted_revenue,predicted_profit,income_am,profit_last_am,profit_am,damage_am,damage_inc,...,claims_am,nights_booked,gender,shop_am,shop_use,retired,gold_status,combined_pos_score,combined_neg_score,net_score
1,1,0.000000,0,2597.720755,2597.720755,3990.0,9.0,3450.0,0.0,0.0,...,0.0,4.0,M,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
3,3,0.000000,0,2496.358789,2496.358789,2451.0,791.0,2119.0,0.0,0.0,...,0.0,3.0,M,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
7,7,0.000000,0,3024.407569,3024.407569,2591.0,869.0,2209.0,0.0,0.0,...,0.0,2.0,M,1415.021157,1.0,1.0,0.0,0.678448,5.867314,-5.188866
8,8,0.000000,0,3490.457498,3490.457498,6426.0,2018.0,9015.0,2333.0,2.0,...,0.0,4.0,M,0.000000,0.0,0.0,0.0,0.576605,0.277479,0.299126
15,15,0.000000,0,2066.260979,2066.260979,1190.0,325.0,6688.0,1676.0,2.0,...,0.0,52.0,V,920.210464,1.0,0.0,0.0,0.411386,0.262099,0.149287
16,16,0.000000,0,3524.023835,3524.023835,927.0,193.0,8747.0,0.0,1.0,...,0.0,4.0,V,1017.002892,1.0,0.0,0.0,0.105045,-0.138518,0.243562
19,19,0.000000,0,2398.516907,2398.516907,227.0,0.0,1620.0,0.0,0.0,...,0.0,1.0,V,0.000000,0.0,0.0,0.0,0.971386,0.679476,0.291909
24,24,0.000000,0,2265.893646,2265.893646,227.0,0.0,1632.0,0.0,0.0,...,0.0,4.0,M,0.000000,0.0,1.0,0.0,0.000000,0.000000,0.000000
30,30,0.000000,0,2363.624904,2363.624904,359.0,844.5,1689.0,0.0,0.0,...,0.0,4.0,V,0.000000,0.0,0.0,0.0,0.378813,0.035107,0.343707
31,31,0.000000,0,4032.381528,4032.381528,14556.0,3440.0,4013.0,0.0,0.0,...,0.0,2.0,V,5568.219879,1.0,1.0,0.0,0.700267,0.399948,0.300319


In [15]:
final_list = highest_profit_unsorted[[
    "applicant_id",
    "predicted_damage_amount",
    "predicted_damage_incident",
    "predicted_revenue",
    "predicted_profit",
]]

final_list_with_matching_row_number = final_list.copy()
final_list_with_matching_row_number["applicant_id"] += 2

In [16]:
final_list.to_csv("./guestlist/guestlist.csv", index=False)
final_list_with_matching_row_number.to_csv("./guestlist/guestlist_with_matching_row_number.csv", index=False)

## **! The applicant_id and index do not match the original row_number in the scores.csv**

They are 2 off, for some reason, e.g. applicant 100 in `highest_profit_unsorted` is actually row 102 in `scores.csv` 

You can tell because the `income_am` of row 102 in `scores.csv` is 122993.0
and in `score` the `income_am` of 100 is 122993.0




In [17]:
# this is row 102 in `scores.csv`
score.iloc[100]["income_am"]

122993.0

![screenshot that shows that index is off by 2](./public/screenshot.png)

### **HOWEVER**

It doesn't really matter.

If you are going by which row number in scores.csv got approved, then you can just subtract 2


If row 3 (with `income_am` of 3990.0) should have been approved, then now it's actually application_id 1

#### **Here are, according to my models, which 200 applicants should be approved**

In [18]:
result = final_list.index.to_numpy()

# np.savetxt("./guestlist/rows/result.csv", result, delimiter=",", fmt="%d")

result

array([  1,   3,   7,   8,  15,  16,  19,  24,  30,  31,  37,  42,  43,
        48,  52,  54,  55,  58,  67,  68,  69,  71,  72,  74,  77,  78,
        81,  84,  85,  86,  89,  91,  94,  95,  98, 100, 102, 103, 105,
       110, 114, 115, 117, 119, 121, 122, 123, 124, 129, 131, 132, 137,
       140, 149, 150, 152, 153, 155, 158, 160, 163, 165, 168, 169, 170,
       174, 175, 177, 179, 181, 184, 188, 189, 190, 191, 196, 199, 201,
       202, 205, 209, 210, 217, 222, 225, 228, 230, 236, 237, 238, 240,
       242, 245, 246, 248, 249, 252, 253, 254, 256, 260, 261, 262, 263,
       264, 268, 269, 270, 271, 272, 274, 275, 277, 279, 280, 284, 288,
       291, 292, 295, 299, 303, 304, 308, 311, 312, 315, 317, 320, 323,
       324, 326, 333, 334, 335, 336, 340, 341, 342, 343, 350, 353, 357,
       358, 359, 361, 371, 372, 373, 378, 379, 386, 388, 390, 392, 393,
       398, 400, 401, 402, 407, 409, 410, 413, 416, 420, 421, 422, 431,
       433, 438, 444, 448, 449, 451, 454, 455, 456, 457, 458, 45

#### **And here they are, with the number matching the row number from `scores.csv`**

In [19]:
result_with_matching_row_number = np.add(final_list.index.to_numpy(), 2)

# np.savetxt("./guestlist/rows/result_with_matching_row_number.csv", result_with_matching_row_number, delimiter=",", fmt="%d")

result_with_matching_row_number

array([  3,   5,   9,  10,  17,  18,  21,  26,  32,  33,  39,  44,  45,
        50,  54,  56,  57,  60,  69,  70,  71,  73,  74,  76,  79,  80,
        83,  86,  87,  88,  91,  93,  96,  97, 100, 102, 104, 105, 107,
       112, 116, 117, 119, 121, 123, 124, 125, 126, 131, 133, 134, 139,
       142, 151, 152, 154, 155, 157, 160, 162, 165, 167, 170, 171, 172,
       176, 177, 179, 181, 183, 186, 190, 191, 192, 193, 198, 201, 203,
       204, 207, 211, 212, 219, 224, 227, 230, 232, 238, 239, 240, 242,
       244, 247, 248, 250, 251, 254, 255, 256, 258, 262, 263, 264, 265,
       266, 270, 271, 272, 273, 274, 276, 277, 279, 281, 282, 286, 290,
       293, 294, 297, 301, 305, 306, 310, 313, 314, 317, 319, 322, 325,
       326, 328, 335, 336, 337, 338, 342, 343, 344, 345, 352, 355, 359,
       360, 361, 363, 373, 374, 375, 380, 381, 388, 390, 392, 394, 395,
       400, 402, 403, 404, 409, 411, 412, 415, 418, 422, 423, 424, 433,
       435, 440, 446, 450, 451, 453, 456, 457, 458, 459, 460, 46

Example:

applicant_id 1 became row 3: the guest with `income_am` of 3990.0

#### **The list of accepted applicants, with the other metrics (the deliverable), can be found in**

`/guestlist/result.csv` and `/guestlist/result_with_matching_row_numbers`